In [ ]:
import $ivy.`org.locationtech.geotrellis::geotrellis-raster:3.5.0`
import $ivy.`org.slf4j:slf4j-simple:1.7.30`

## RasterSource for Landsat
- [RasterSource Overview](https://geotrellis.github.io/geotrellis-workshop/docs/rastersource)
- [RasterSource ScalaDoc](https://geotrellis.github.io/scaladocs/latest/geotrellis/raster/RasterSource.html)

In [ ]:
import geotrellis.raster._
import geotrellis.raster.geotiff.GeoTiffRasterSource

def assetUri(key: String) = s"https://geotrellis-workshop.s3.amazonaws.com/$key"
def bandUri(band: String) = assetUri(s"landsat/LC81070352015218LGN00_$band.TIF")

val greenBand = GeoTiffRasterSource(bandUri("B3"))
val redBand   = GeoTiffRasterSource(bandUri("B4"))
val nirBand   = GeoTiffRasterSource(bandUri("B5"))
val qaBand    = GeoTiffRasterSource(bandUri("BQA"))

In [ ]:
redBand.metadata

## Read Overview

Landsat scenes in the `geotrellis-workshop` bucket have added overviews

- `RasterSource.resolutions`
- `RasterSource.resample`
- Reading tiles, `Option` return
- Rendering tiles
- Fixing `NODATA` value

In [ ]:
import geotrellis.raster.resample._
import geotrellis.raster.io.geotiff._

val overview = redBand.resample(
    resampleTarget = TargetCellSize(CellSize(500,500)), 
    method = NearestNeighbor, 
    strategy = Auto())

val tile = overview.read().get.tile.band(0)

In [ ]:
val histogram = tile.histogram
val colorMap = ColorRamps.BlueToRed.toColorMap(histogram)

Image(tile.renderPng(colorMap).bytes)

## Read GeoJSON

In [ ]:
import geotrellis.vector._
val json = scala.io.Source.fromURL(assetUri("gadm36/JPN_1_Chiba.geojson")).mkString

In [ ]:
val chibaAoi = json.parseJson.as[MultiPolygon].right.get

## Read Window from Landsat

- [MultiPolygon Reproject ScalaDoc](https://geotrellis.github.io/scaladocs/latest/geotrellis/vector/reproject/Implicits$ReprojectMutliPolygon.html)
- [MultiPolygon Reproject Implicit Method](https://github.com/locationtech/geotrellis/blob/2f8348ac299d889282b7e6d379eed4696ece1dd7/vector/src/main/scala/geotrellis/vector/reproject/Implicits.scala#L89)                                   

In [ ]:
greenBand.read(chibaAoi.extent) // Oh no, I should have the data!

In [ ]:
import geotrellis.proj4._
val chibaAoiUtm = chibaAoi.reproject(LatLng, greenBand.crs)

In [ ]:
val chibaRedRaster = redBand.read(chibaAoiUtm.extent).get

In [ ]:
val chibaRedBand = chibaRedRaster.tile.band(0).withNoData(Some(0))
Image(chibaRedBand.renderPng(colorMap).bytes)

## Rasterize AOI

In [ ]:
val chibaMask = chibaRedBand.mutable

chibaGreenRaster.rasterExtent.foreach(chibaAoiUtm) { (x, y) =>
    chibaMask.set(x, y, Short.MaxValue)
}

// this will work, but we just mutated a Tile!
Image(chibaMask.renderPng(colorMap).bytes)

## Mask Clouds using QA Layer

In [ ]:
val qaTile = qaBand.read(chibaAoiUtm.extent).get.tile.band(0).withNoData(Some(0))

def maskClouds(tile: Tile): Tile =
  tile.combine(qaTile) { (v: Int, qa: Int) =>
    val isCloud = qa & 0x8000
    val isCirrus = qa & 0x2000
    if(isCloud > 0 || isCirrus > 0) { NODATA }
    else { v }
  }

In [ ]:
Image(maskClouds(chibaRedBand).renderPng(colorMap).bytes)

## Compute NDVI

In [ ]:
def ndvi (r: Double, ir: Double) : Double = {
    if (isData(r) && isData(ir)) {
        (ir - r) / (ir + r)
    } else {
      Double.NaN
    }
}
val chibaNirBand = nirBand.read(chibaAoiUtm.extent).get.tile.band(0)

val red = maskClouds(chibaRedBand).convert(DoubleConstantNoDataCellType)
val nir = maskClouds(chibaNirBand).convert(DoubleConstantNoDataCellType)

val chibaNdvi = red.combineDouble(nir) { (r: Double, ir: Double) =>
    if (isData(r) && isData(ir)) {
        (ir - r) / (ir + r)
    } else {
      Double.NaN
    }
}

val ndviColorMap = ColorMap.fromStringDouble("0:ffffe5ff;0.1:f7fcb9ff;0.2:d9f0a3ff;0.3:addd8eff;0.4:78c679ff;0.5:41ab5dff;0.6:238443ff;0.7:006837ff;1:004529ff").get
Image(chibaNdvi.renderPng(ndviColorMap).bytes)

In [ ]:
GeoTiff(chibaNdvi, chibaRedRaster.extent, redBand.crs).write("ndvi.tif")